#랜덤포레스트 다중분류
- adult dataset
- 가격이 50만 이상인지 이하인지 예측하는 문제

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

##1.데이터 준비

In [2]:
! wget https://raw.githubusercontent.com/devdio/datasets/main/adult.csv

--2024-01-04 06:36:24--  https://raw.githubusercontent.com/devdio/datasets/main/adult.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5326368 (5.1M) [text/plain]
Saving to: ‘adult.csv.1’

adult.csv.1         100%[===================>]   5.08M  --.-KB/s    in 0.08s   

2024-01-04 06:36:24 (64.2 MB/s) - ‘adult.csv.1’ saved [5326368/5326368]



In [3]:
adult = pd.read_csv('adult.csv')

In [4]:
df = adult

In [5]:
df.shape

(48842, 15)

In [6]:
df.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        48842 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   education        48842 non-null  object
 4   educational-num  48842 non-null  int64 
 5   marital-status   48842 non-null  object
 6   occupation       48842 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   gender           48842 non-null  object
 10  capital-gain     48842 non-null  int64 
 11  capital-loss     48842 non-null  int64 
 12  hours-per-week   48842 non-null  int64 
 13  native-country   48842 non-null  object
 14  income           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [8]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
age,48842.0,38.643585,13.710510,17.0,28.0,37.0,48.0,90.0
fnlwgt,48842.0,189664.134597,105604.025423,12285.0,117550.5,178144.5,237642.0,1490400.0
educational-num,48842.0,10.078089,2.570973,1.0,9.0,10.0,12.0,16.0
capital-gain,48842.0,1079.067626,7452.019058,0.0,0.0,0.0,0.0,99999.0
capital-loss,48842.0,87.502314,403.004552,0.0,0.0,0.0,0.0,4356.0
hours-per-week,48842.0,40.422382,12.391444,1.0,40.0,40.0,45.0,99.0


In [9]:
df=df.replace('?',np.nan)

In [10]:
df.isna().sum()

age                   0
workclass          2799
fnlwgt                0
education             0
educational-num       0
marital-status        0
occupation         2809
relationship          0
race                  0
gender                0
capital-gain          0
capital-loss          0
hours-per-week        0
native-country      857
income                0
dtype: int64

In [11]:
df.head(10)

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,<=50K
5,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K
6,29,NaN,227026,HS-grad,9,Never-married,NaN,Unmarried,Black,Male,0,0,40,United-States,<=50K
7,63,Self-emp-not-inc,104626,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,3103,0,32,United-States,>50K
8,24,Private,369667,Some-college,10,Never-married,Other-service,Unmarried,White,Female,0,0,40,United-States,<=50K
9,55,Private,104996,7th-8th,4,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,10,United-States,<=50K


In [12]:
df['workclass'].value_counts()

Private             33906
Self-emp-not-inc     3862
Local-gov            3136
State-gov            1981
Self-emp-inc         1695
Federal-gov          1432
Without-pay            21
Never-worked           10
Name: workclass, dtype: int64

In [13]:
df['occupation'].value_counts()

Prof-specialty       6172
Craft-repair         6112
Exec-managerial      6086
Adm-clerical         5611
Sales                5504
Other-service        4923
Machine-op-inspct    3022
Transport-moving     2355
Handlers-cleaners    2072
Farming-fishing      1490
Tech-support         1446
Protective-serv       983
Priv-house-serv       242
Armed-Forces           15
Name: occupation, dtype: int64

In [14]:
df=df.dropna()

In [15]:
df.isna().sum()

age                0
workclass          0
fnlwgt             0
education          0
educational-num    0
marital-status     0
occupation         0
relationship       0
race               0
gender             0
capital-gain       0
capital-loss       0
hours-per-week     0
native-country     0
income             0
dtype: int64

In [16]:
df.shape

(45222, 15)

In [17]:
df=df.drop(['education'],axis=1)

In [18]:
df=df.drop(['native-country'],axis=1)

##3.테스트 데이터 만들기

In [19]:
df.head()

,age,workclass,fnlwgt,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,income
0,25,Private,226802,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,<=50K
1,38,Private,89814,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,<=50K
2,28,Local-gov,336951,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,>50K
3,44,Private,160323,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,>50K
5,34,Private,198693,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,<=50K


In [20]:
X= df.drop("income",axis=1)
Y= df['income']

In [21]:
df['income'].value_counts()

<=50K    34014
>50K     11208
Name: income, dtype: int64

In [22]:
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test=train_test_split(X,Y, test_size=0.3, stratify=Y,random_state=0)

In [23]:
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

((31655, 12), (13567, 12), (31655,), (13567,))

##4.데이터 전처리



In [24]:
X_train.head()

,age,workclass,fnlwgt,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week
44856,29,Private,412435,9,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40
8187,51,Self-emp-not-inc,85877,9,Divorced,Exec-managerial,Not-in-family,White,Female,99999,0,60
26304,49,Private,105449,13,Never-married,Priv-house-serv,Not-in-family,White,Male,0,0,25
35153,31,Private,209538,10,Married-civ-spouse,Adm-clerical,Husband,White,Male,0,0,55
35505,25,Private,521400,3,Never-married,Machine-op-inspct,Other-relative,White,Male,0,0,40


In [25]:
df.columns

Index(['age', 'workclass', 'fnlwgt', 'educational-num', 'marital-status',
       'occupation', 'relationship', 'race', 'gender', 'capital-gain',
       'capital-loss', 'hours-per-week', 'income'],
      dtype='object')

In [26]:
cat_cols=['workclass','marital-status','occupation', 'relationship', 'race', 'gender']   #원핫
num_cols=['age', 'fnlwgt', 'educational-num','capital-gain', 'capital-loss', 'hours-per-week']      # 스케일링

In [27]:
from sklearn.preprocessing import OneHotEncoder

oe = OneHotEncoder(sparse_output=False)                #sparse_output=False 을 함으로써 True를 하면 우리가 알던 원핫 인코딩인데 공간과 보기가 힘들어서 False 를 해서 보기 편하게 만든다
X_train_cat=oe.fit_transform(X_train[cat_cols])
X_train_cat

array([[0., 0., 1., ..., 1., 1., 0.],
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 1., ..., 1., 0., 1.],
       ...,
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 1., ..., 1., 0., 1.],
       [0., 0., 1., ..., 1., 1., 0.]])

In [28]:
from sklearn.preprocessing import StandardScaler

ss=StandardScaler()
X_train_num=ss.fit_transform(X_train[num_cols])
X_train_num

array([[-0.72242557,  2.11585983, -0.43990255, -0.14753966, -0.21767013,
        -0.08183881],
       [ 0.94555971, -0.98965713, -0.43990255, 13.02131581, -0.21767013,
         1.58457545],
       [ 0.79392468, -0.80353038,  1.13302361, -0.14753966, -0.21767013,
        -1.33164951],
       ...,
       [-0.57079055, -0.75346103, -0.04667101, -0.14753966, -0.21767013,
        -0.33180095],
       [ 0.41483712, -0.50361831, -0.04667101, -0.14753966, -0.21767013,
        -0.08183881],
       [-0.19170298, -0.57616894, -0.04667101, -0.14753966, -0.21767013,
        -0.08183881]])

In [29]:
X_train_s=np.concatenate((X_train_cat,X_train_num),axis=1)
X_train_s

array([[ 0.        ,  0.        ,  1.        , ..., -0.14753966,
        -0.21767013, -0.08183881],
       [ 0.        ,  0.        ,  0.        , ..., 13.02131581,
        -0.21767013,  1.58457545],
       [ 0.        ,  0.        ,  1.        , ..., -0.14753966,
        -0.21767013, -1.33164951],
       ...,
       [ 0.        ,  0.        ,  0.        , ..., -0.14753966,
        -0.21767013, -0.33180095],
       [ 0.        ,  0.        ,  1.        , ..., -0.14753966,
        -0.21767013, -0.08183881],
       [ 0.        ,  0.        ,  1.        , ..., -0.14753966,
        -0.21767013, -0.08183881]])

In [30]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
Y_train=le.fit_transform(Y_train)
Y_train

array([0, 1, 0, ..., 0, 0, 0])

In [31]:
type(X_train_s),type(Y_train)

(numpy.ndarray, numpy.ndarray)

##5.학습

##### 테스트 데이터도 전처리 해주기




In [32]:
X_test

,age,workclass,fnlwgt,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week
36652,50,Private,410186,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40
10163,23,Private,85139,9,Never-married,Adm-clerical,Not-in-family,White,Female,0,0,40
30069,57,State-gov,32694,16,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40
33245,45,Local-gov,209482,14,Married-civ-spouse,Prof-specialty,Wife,White,Female,0,0,45
19011,54,Private,96710,9,Married-civ-spouse,Priv-house-serv,Other-relative,Black,Female,0,0,20
...,...,...,...,...,...,...,...,...,...,...,...,...
7736,30,Private,90308,1,Never-married,Other-service,Unmarried,White,Male,0,0,28
10979,48,State-gov,98010,14,Married-civ-spouse,Prof-specialty,Wife,White,Female,0,0,16
4653,42,Private,244668,9,Divorced,Tech-support,Not-in-family,White,Male,8614,0,40
5929,46,Local-gov,258498,14,Divorced,Prof-specialty,Unmarried,White,Female,0,0,40


In [33]:
X_test_cat=oe.fit_transform(X_test[cat_cols])
X_test_cat

array([[0., 0., 1., ..., 1., 0., 1.],
       [0., 0., 1., ..., 1., 1., 0.],
       [0., 0., 0., ..., 1., 0., 1.],
       ...,
       [0., 0., 1., ..., 1., 0., 1.],
       [0., 1., 0., ..., 1., 1., 0.],
       [0., 0., 1., ..., 0., 0., 1.]])

In [34]:
X_test_num=ss.fit_transform(X_test[num_cols])
X_test_num

array([[ 0.85873501,  2.06954753, -0.43404361, -0.14486082, -0.22135501,
        -0.06946042],
       [-1.17392561, -0.97519222, -0.43404361, -0.14486082, -0.22135501,
        -0.06946042],
       [ 1.38572109, -1.46644849,  2.28374419, -0.14486082, -0.22135501,
        -0.06946042],
       ...,
       [ 0.25646519,  0.51912795, -0.43404361,  1.03537788, -0.22135501,
        -0.06946042],
       [ 0.5576001 ,  0.6486746 ,  1.50723339, -0.14486082, -0.22135501,
        -0.06946042],
       [-0.5716558 , -1.43675486,  1.11897799, -0.14486082, -0.22135501,
         1.59447444]])

In [35]:
X_test_s=np.concatenate((X_test_cat,X_test_num),axis=1)
X_test_s

array([[ 0.        ,  0.        ,  1.        , ..., -0.14486082,
        -0.22135501, -0.06946042],
       [ 0.        ,  0.        ,  1.        , ..., -0.14486082,
        -0.22135501, -0.06946042],
       [ 0.        ,  0.        ,  0.        , ..., -0.14486082,
        -0.22135501, -0.06946042],
       ...,
       [ 0.        ,  0.        ,  1.        , ...,  1.03537788,
        -0.22135501, -0.06946042],
       [ 0.        ,  1.        ,  0.        , ..., -0.14486082,
        -0.22135501, -0.06946042],
       [ 0.        ,  0.        ,  1.        , ..., -0.14486082,
        -0.22135501,  1.59447444]])

In [36]:
Y_test=le.fit_transform(Y_test)
Y_test

array([0, 0, 1, ..., 1, 0, 0])

In [37]:
type(X_test_s), type(Y_test)

(numpy.ndarray, numpy.ndarray)

##### 의사결정나무

In [38]:
from sklearn.tree import DecisionTreeClassifier

clf=DecisionTreeClassifier()
clf.fit(X_train_s,Y_train)

DecisionTreeClassifier()

In [39]:
X_test_s.shape, X_train_s.shape

((13567, 47), (31655, 47))

In [40]:
Y_pred =clf.predict(X_test_s)
Y_pred

array([1, 0, 1, ..., 1, 0, 1])

In [41]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
def print_metrics(Y_test,Y_pred):
  acc= accuracy_score(Y_test,Y_pred)
  re= recall_score(Y_test,Y_pred)
  pr= precision_score(Y_test,Y_pred)
  f1= f1_score(Y_test,Y_pred)
  print('정확도:',acc)
  print('재현율:',re)
  print('정밀도:',pr)
  print('f1스코어:',f1)
print_metrics(Y_test,Y_pred)

정확도: 0.8038623129652834
재현율: 0.6157049375371803
정밀도: 0.6019191625472521
f1스코어: 0.6087340097044551


In [42]:
from sklearn.model_selection import GridSearchCV

grid_params = {
    "max_depth":[2,3,4],
    'max_leaf_nodes':[5,6],
    'criterion':["gini","entropy","log_loss"]
}

In [43]:
clf = DecisionTreeClassifier()
gs = GridSearchCV(clf, grid_params, cv=5,verbose=5)
gs.fit(X_train_s, Y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV 1/5] END criterion=gini, max_depth=2, max_leaf_nodes=5;, score=0.831 total time=   0.1s
[CV 2/5] END criterion=gini, max_depth=2, max_leaf_nodes=5;, score=0.822 total time=   0.1s
[CV 3/5] END criterion=gini, max_depth=2, max_leaf_nodes=5;, score=0.820 total time=   0.1s
[CV 4/5] END criterion=gini, max_depth=2, max_leaf_nodes=5;, score=0.826 total time=   0.2s
[CV 5/5] END criterion=gini, max_depth=2, max_leaf_nodes=5;, score=0.825 total time=   0.2s
[CV 1/5] END criterion=gini, max_depth=2, max_leaf_nodes=6;, score=0.831 total time=   0.2s
[CV 2/5] END criterion=gini, max_depth=2, max_leaf_nodes=6;, score=0.822 total time=   0.1s
[CV 3/5] END criterion=gini, max_depth=2, max_leaf_nodes=6;, score=0.820 total time=   0.1s
[CV 4/5] END criterion=gini, max_depth=2, max_leaf_nodes=6;, score=0.826 total time=   0.1s
[CV 5/5] END criterion=gini, max_depth=2, max_leaf_nodes=6;, score=0.825 total time=   0.2s
[CV 1/5] END criter

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': [2, 3, 4], 'max_leaf_nodes': [5, 6]},
             verbose=5)

In [44]:
gs.best_params_

{'criterion': 'entropy', 'max_depth': 3, 'max_leaf_nodes': 5}

In [45]:
gs.best_score_

0.8403727689148635

In [46]:
clf_t = DecisionTreeClassifier(criterion= 'entropy', max_depth= 3, max_leaf_nodes= 5)
clf_t.fit(X_train_s,Y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=3, max_leaf_nodes=5)

#####랜덤포레스트

```
# 코드로 형식 지정됨
```



In [47]:
from sklearn.ensemble import RandomForestClassifier

clf=RandomForestClassifier()
clf.fit(X_train_s,Y_train)


RandomForestClassifier()

In [48]:
Y_pred =clf.predict(X_test_s)
Y_pred

array([1, 0, 1, ..., 1, 0, 0])

In [49]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
def print_metrics(Y_test,Y_pred):
  acc= accuracy_score(Y_test,Y_pred)
  re= recall_score(Y_test,Y_pred)
  pr= precision_score(Y_test,Y_pred)
  f1= f1_score(Y_test,Y_pred)
  print('정확도:',acc)
  print('재현율:',re)
  print('정밀도:',pr)
  print('f1스코어:',f1)
print_metrics(Y_test,Y_pred)

정확도: 0.8439596078720425
재현율: 0.6103509815585961
정밀도: 0.7177334732423925
f1스코어: 0.659701012698923


In [50]:
from sklearn.model_selection import GridSearchCV

grid_params = {
    "max_depth":[13,16,19],
    'n_estimators':[80,100,120]
}

In [51]:
gs = GridSearchCV(clf, grid_params, cv=5,verbose=5)
gs.fit(X_train_s, Y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5] END .....max_depth=13, n_estimators=80;, score=0.864 total time=   1.5s
[CV 2/5] END .....max_depth=13, n_estimators=80;, score=0.857 total time=   1.4s
[CV 3/5] END .....max_depth=13, n_estimators=80;, score=0.858 total time=   1.4s
[CV 4/5] END .....max_depth=13, n_estimators=80;, score=0.864 total time=   1.4s
[CV 5/5] END .....max_depth=13, n_estimators=80;, score=0.861 total time=   1.4s
[CV 1/5] END ....max_depth=13, n_estimators=100;, score=0.864 total time=   1.8s
[CV 2/5] END ....max_depth=13, n_estimators=100;, score=0.857 total time=   2.4s
[CV 3/5] END ....max_depth=13, n_estimators=100;, score=0.858 total time=   2.6s
[CV 4/5] END ....max_depth=13, n_estimators=100;, score=0.864 total time=   1.7s
[CV 5/5] END ....max_depth=13, n_estimators=100;, score=0.863 total time=   1.8s
[CV 1/5] END ....max_depth=13, n_estimators=120;, score=0.864 total time=   2.1s
[CV 2/5] END ....max_depth=13, n_estimators=120;,

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [13, 16, 19],
                         'n_estimators': [80, 100, 120]},
             verbose=5)

In [52]:
gs.best_params_

{'max_depth': 19, 'n_estimators': 100}

In [53]:
gs.best_score_

0.8630864002527247

In [54]:
clf_r= RandomForestClassifier(max_depth= 19, n_estimators= 80)
clf_r.fit(X_train_s,Y_train)

RandomForestClassifier(max_depth=19, n_estimators=80)

#####SVM

In [55]:
from sklearn.svm import SVC

clf = SVC()
clf.fit(X_train_s,Y_train)

SVC()

In [56]:
Y_pred=clf.predict(X_test_s)
Y_pred

array([0, 0, 1, ..., 1, 0, 0])

In [57]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
def print_metrics(Y_test,Y_pred):
  acc= accuracy_score(Y_test,Y_pred)
  re= recall_score(Y_test,Y_pred)
  pr= precision_score(Y_test,Y_pred)
  f1= f1_score(Y_test,Y_pred)
  print('정확도:',acc)
  print('재현율:',re)
  print('정밀도:',pr)
  print('f1스코어:',f1)
print_metrics(Y_test,Y_pred)

정확도: 0.8517726837178448
재현율: 0.6055919095776323
정밀도: 0.7482543182653436
f1스코어: 0.6694065428242644


In [58]:
from sklearn.model_selection import GridSearchCV

grid_params = {
    "C":[1,2],
    'kernel':['linear','poly'],
    'gamma':[0.1,0.5]
}

In [59]:
clf = SVC()
gs = GridSearchCV(clf, grid_params, cv=2,verbose=2)
gs.fit(X_train_s, Y_train)

Fitting 2 folds for each of 8 candidates, totalling 16 fits
[CV] END ......................C=1, gamma=0.1, kernel=linear; total time=  15.0s
[CV] END ......................C=1, gamma=0.1, kernel=linear; total time=  15.6s
[CV] END ........................C=1, gamma=0.1, kernel=poly; total time=  14.3s
[CV] END ........................C=1, gamma=0.1, kernel=poly; total time=  15.1s
[CV] END ......................C=1, gamma=0.5, kernel=linear; total time=  15.8s
[CV] END ......................C=1, gamma=0.5, kernel=linear; total time=  15.4s
[CV] END ........................C=1, gamma=0.5, kernel=poly; total time= 2.6min
[CV] END ........................C=1, gamma=0.5, kernel=poly; total time= 3.3min
[CV] END ......................C=2, gamma=0.1, kernel=linear; total time=  17.1s
[CV] END ......................C=2, gamma=0.1, kernel=linear; total time=  18.4s
[CV] END ........................C=2, gamma=0.1, kernel=poly; total time=  15.6s
[CV] END ........................C=2, gamma=0.1, 

GridSearchCV(cv=2, estimator=SVC(),
             param_grid={'C': [1, 2], 'gamma': [0.1, 0.5],
                         'kernel': ['linear', 'poly']},
             verbose=2)

In [60]:
gs.best_params_

{'C': 1, 'gamma': 0.1, 'kernel': 'poly'}

In [61]:
gs.best_score_

0.8499763218802544

In [62]:
clf_s = SVC(C=1,gamma=0.1,kernel='poly')
clf_s.fit(X_train_s,Y_train)

SVC(C=1, gamma=0.1, kernel='poly')

##6.모델저장

In [63]:
import pickle

with open('rd-bin-adult-tree','wb') as file:
  pickle.dump(clf_t,file)

In [64]:
with open('rd-bin-adult-random','wb') as file:
  pickle.dump(clf_r,file)

In [65]:
with open('rd-bin-adult-svm','wb') as file:
  pickle.dump(clf_s,file)

In [66]:
with open('rd-bin-adult-oe','wb') as file:
  pickle.dump(oe,file)

In [67]:
with open('rd-bin-adult-le','wb') as file:
  pickle.dump(le,file)

In [68]:
with open('rd-bin-adult-ss','wb') as file:
  pickle.dump(ss,file)